## 수정사항
#### 2021.06.04
* Gripper 자유도 45도로 제한됨에 따라 파지점 방향 수정, 바닥 파지면은 삭제

## set running directory

In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir


DATA_PATH = os.path.join(os.environ['RNB_PLANNING_DIR'], "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "test")
try_mkdir(TEST_DATA_PATH)

In [3]:
VISUALIZE = False
TIMEOUT_REACH = 30
TIMEOUT_RETRIEVE = 30
TIMEOUT_SELF = 3
TOOL_RPY = (-np.pi/2,0,0)
SHOW_PERIOD = 0.01

ROBOT_TYPE = RobotType.panda

if ROBOT_TYPE == RobotType.indy7:
    ROBOT_NAME = "indy0"
    TOOL_LINK = "indy0_tcp"
    TOOL_XYZ = (0,0,0.14)
    GRIP_DEPTH = 0.05
    HOME_POSE = (0,0,0,0,0,0)
elif ROBOT_TYPE == RobotType.panda:
    ROBOT_NAME = "panda0"
    TOOL_LINK = "panda0_hand"
    TOOL_XYZ = (0,0,0.112)
    GRIP_DEPTH = 0.03
    HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)
else:
    raise(NotImplementedError("Unknown robot type"))
    
    
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, ROBOT_TYPE.name)
try_mkdir(ROBOT_DATA_ROOT)

## init combined robot config

In [4]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                INDY_IP)]
              , connection_list=[False])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

connection command:
panda0: False


## get ghnd with detected robot config

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


In [6]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [7]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

## init planning scene

In [8]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [9]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [10]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                 dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0))

## planner

In [11]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker


gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
lcheck = LatticedChecker(pscene, gcheck)

g_checker = MoveitPlanner(pscene, motion_filters = [gcheck])
gr_checker = MoveitPlanner(pscene, motion_filters = [gcheck, rcheck])
grl_checker = MoveitPlanner(pscene, motion_filters = [gcheck, rcheck, lcheck])

## ui

In [12]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


# Object Classes

In [13]:
from pkg.planning.filtering.lattice_model import *
from pkg.planning.filtering.lattice_model.scene_building import *
        
WORKPLANE_TYPES = [WorkPlane, Box, SideBox, TopBox]

Nmax_wall, Nmax_pole, Nmax_bar = 4, 8, 8

Nmax_obj = 3

N_max_sample = 100
N_print = 5
N_sample_single_obj = 3

   Use a production WSGI server instead.
 * Debug mode: off


# Load and verify

In [14]:
N_MAX = 1000


gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
DATASET_LIST = sorted(os.listdir(ROBOT_DATA_ROOT))
N_tot = 0
N_suc = 0
labels = []
g_results = []
gr_results = []
grl_results = []
print("DATASET_LIST: {}".format(DATASET_LIST))
gtimer.tic("full_loop")
for DATASET_DIR in DATASET_LIST:
    DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DATASET_DIR)
    print("DATASET_PATH: {}".format(DATASET_PATH))
    DATA_LIST = sorted(os.listdir(DATASET_PATH))
    print("DATA_LIST: {}".format(DATA_LIST))
    for DATA_FILE in DATA_LIST:
        DATA_PATH = os.path.join(DATASET_PATH, DATA_FILE)
        print("DATA_PATH: {}".format(DATA_PATH))
        data = load_pickle(DATA_PATH)
        gtem_args = data["gtem_args"]
        obj_args = data["obj_args"]
        samples = data["samples"]
        gtem_remove = []
        for gtem in gscene:
            if gtem.link_name == "base_link" and gtem.parent is None:
                gtem_remove.append(gtem)
        for gtem in gtem_remove:
            gscene.remove(gtem)
                
        gid_list = np.arange(len(gtem_args)).tolist()
        for gidx in gid_list:
            args = gtem_args[gidx]
            if args['parent'] is not None:
                if args['parent'] not in gscene.NAME_DICT:
                    gid_list.append(gidx)
                    continue
            gscene.create_safe(**args)
            
        pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)
            
        for i_s, sample in enumerate(samples):
            obj_name, from_state, to_state, redundancy_dict, \
                success_kin_, success_reach_, success_retrieve_, \
                Traj_kin_, Traj_reach_, Traj_retrieve_, \
                time_kin_, time_reach_, time_retrieve_ = sample
            
            obj, obj_arg = DummyObject(), obj_args[obj_name]
            obj.name = obj_name
            obj.GRIP_DEPTH = obj_arg["GRIP_DEPTH"]
            obj.DIM = obj_arg["DIM"]
            obj.geometry = gscene.NAME_DICT[obj_arg["gname"]]
            obj_pscene, handles = add_object(pscene, obj)
            
            gscene.update()
#             g_checker.update_gscene(only_self_collision=True)
#             gr_checker.update_gscene(only_self_collision=True)
#             grl_checker.update_gscene(only_self_collision=True)
                            
            pscene.set_object_state(from_state)

            with gtimer.block("Grasp"):
                success_g = \
                    g_checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                               test_filters_only=True, timeout=TIMEOUT_SELF)

            with gtimer.block("GraspReach"):
                success_gr = \
                    gr_checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                               test_filters_only=True, timeout=TIMEOUT_SELF)

            with gtimer.block("GRL"):
                success_grl = \
                    grl_checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                               test_filters_only=True, timeout=TIMEOUT_SELF)

            labels.append(success_retrieve_)
            g_results.append(success_g)
            gr_results.append(success_gr)
            grl_results.append(success_grl)
                
            pscene.set_object_state(from_state)
            pscene.remove_subject(obj_pscene.oname)
            for handle in handles:
                gscene.remove(handle)  
            gscene.update_markers_all()  
            N_tot += 1
            N_suc += success_retrieve_
            if N_tot>=N_MAX:
                break
        print("GRL: {} / SUCCESS: {} / TOTAL: {} / ELAPSED TIME: {} s / ETA: {} s".format(
            np.sum(grl_results), N_suc, N_tot, *np.round(np.array(gtimer.eta("full_loop", N_tot, N_MAX))/1000, 1)))
        if N_tot>=N_MAX:
            break
    if N_tot>=N_MAX:
        break

DATASET_LIST: ['20210611-071851', '20210611-072450', '20210611-072703', '20210611-072931', '20210611-073335', '20210611-073528', '20210611-073814', '20210611-074130', '20210611-074423', '20210611-074711']
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-071851
DATA_LIST: ['00001.pkl', '00002.pkl', '00003.pkl', '00004.pkl', '00005.pkl', '00006.pkl', '00007.pkl', '00008.pkl', '00009.pkl', '00010.pkl', '00011.pkl', '00012.pkl', '00013.pkl', '00014.pkl', '00015.pkl', '00016.pkl', '00017.pkl', '00018.pkl', '00019.pkl', '00020.pkl', '00021.pkl']
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-071851/00001.pkl
GRL: 0 / SUCCESS: 0 / TOTAL: 3 / ELAPSED TIME: 0.0 s / ETA: 6.0 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-071851/00002.pkl
GRL: 0 / SUCCESS: 0 / TOTAL: 9 / ELAPSED TIME: 0.0 s / ETA: 5.0 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-071851/00003.pkl
GRL: 0 / SUCCESS: 0 / TOTAL: 15 / ELAPSED TIM

GRL: 0 / SUCCESS: 6 / TOTAL: 258 / ELAPSED TIME: 2.7 s / ETA: 10.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-072703/00011.pkl
GRL: 0 / SUCCESS: 6 / TOTAL: 264 / ELAPSED TIME: 2.7 s / ETA: 10.3 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-072703/00012.pkl
GRL: 0 / SUCCESS: 6 / TOTAL: 267 / ELAPSED TIME: 2.8 s / ETA: 10.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-072703/00013.pkl
GRL: 0 / SUCCESS: 6 / TOTAL: 273 / ELAPSED TIME: 2.8 s / ETA: 10.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-072703/00014.pkl
GRL: 0 / SUCCESS: 6 / TOTAL: 279 / ELAPSED TIME: 2.9 s / ETA: 10.2 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-072703/00015.pkl
GRL: 0 / SUCCESS: 6 / TOTAL: 282 / ELAPSED TIME: 2.9 s / ETA: 10.2 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-072703/00016.pkl
GRL: 0 / SUCCESS: 6 / TOTAL: 285 / ELAPSED TIME: 2.9 s / ETA: 10.2 s
DATA_PA

GRL: 1 / SUCCESS: 14 / TOTAL: 504 / ELAPSED TIME: 5.3 s / ETA: 10.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-073335/00018.pkl
GRL: 1 / SUCCESS: 14 / TOTAL: 507 / ELAPSED TIME: 5.3 s / ETA: 10.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-073335/00019.pkl
GRL: 1 / SUCCESS: 14 / TOTAL: 510 / ELAPSED TIME: 5.3 s / ETA: 10.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-073335/00020.pkl
GRL: 1 / SUCCESS: 15 / TOTAL: 516 / ELAPSED TIME: 5.4 s / ETA: 10.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-073335/00021.pkl
GRL: 2 / SUCCESS: 16 / TOTAL: 519 / ELAPSED TIME: 5.4 s / ETA: 10.5 s
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-073528
DATA_LIST: ['00001.pkl', '00002.pkl', '00003.pkl', '00004.pkl', '00005.pkl', '00006.pkl', '00007.pkl', '00008.pkl', '00009.pkl', '00010.pkl', '00011.pkl', '00012.pkl', '00013.pkl', '00014.pkl', '00015.pkl', '00016.pkl', '00017.pkl',

GRL: 6 / SUCCESS: 29 / TOTAL: 756 / ELAPSED TIME: 8.2 s / ETA: 10.8 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-074130/00007.pkl
GRL: 6 / SUCCESS: 29 / TOTAL: 759 / ELAPSED TIME: 8.2 s / ETA: 10.8 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-074130/00008.pkl
GRL: 6 / SUCCESS: 29 / TOTAL: 765 / ELAPSED TIME: 8.2 s / ETA: 10.8 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-074130/00009.pkl
GRL: 6 / SUCCESS: 29 / TOTAL: 774 / ELAPSED TIME: 8.3 s / ETA: 10.7 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-074130/00010.pkl
GRL: 6 / SUCCESS: 29 / TOTAL: 777 / ELAPSED TIME: 8.3 s / ETA: 10.7 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-074130/00011.pkl
GRL: 6 / SUCCESS: 30 / TOTAL: 786 / ELAPSED TIME: 8.4 s / ETA: 10.7 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test/panda/20210611-074130/00012.pkl
GRL: 6 / SUCCESS: 30 / TOTAL: 789 / ELAPSED TIME: 8.5 s / ETA: 10.7 s


In [15]:
print(gtimer)
print("")
print("=========== GraspChecker ==============")
print_confusion_mat(GT=labels, Res=g_results)
print("")
print("=========== GraspReacher ==============")
print_confusion_mat(GT=labels, Res=gr_results)
print("")
print("=========== TotalChecker ==============")
print_confusion_mat(GT=labels, Res=grl_results)

full_loop: 	1099226.8 ms/199 = 5523.8 ms (17.926/11881.761)
Grasp: 	1682.3 ms/1000 = 1.7 ms (0.872/6.219)
GraspReach: 	1809.9 ms/1000 = 1.8 ms (0.929/6.107)
GRL: 	6625.4 ms/1000 = 6.6 ms (0.899/81.159)


=========== GraspChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 53 	 	 0 	 	 100.00%
GN 	 257 	 	 690 	 72.86%
AL 	 17.10% 	 100.00% 	 74.30%

=========== GraspReacher ==============
	 PP 	 	 PN 	 	 1000
GP 	 53 	 	 0 	 	 100.00%
GN 	 38 	 	 909 	 95.99%
AL 	 58.24% 	 100.00% 	 96.20%

=========== TotalChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 15 	 	 38 	 	 28.30%
GN 	 0 	 	 947 	 100.00%
AL 	 100.00% 	 96.14% 	 96.20%


# Indy7
```
=========== GraspChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 51 	 	 0 	 	 100.00%
GN 	 198 	 	 751 	 79.14%
AL 	 20.48% 	 100.00% 	 80.20%
=========== GraspReacher ==============
	 PP 	 	 PN 	 	 1000
GP 	 51 	 	 0 	 	 100.00%
GN 	 16 	 	 933 	 98.31%
AL 	 76.12% 	 100.00% 	 98.40%
=========== FailMore ==============
	 PP 	 	 PN 	 	 1000
GP 	 16 	 	 35 	 	 31.37%
GN 	 2 	 	 947 	 99.79%
AL 	 88.89% 	 96.44% 	 96.30%
=========== Balanced ==============
	 PP 	 	 PN 	 	 1000
GP 	 32 	 	 19 	 	 62.75%
GN 	 6 	 	 943 	 99.37%
AL 	 84.21% 	 98.02% 	 97.50%
=========== Filtered ==============
	 PP 	 	 PN 	 	 1000
GP 	 47 	 	 4 	 	 92.16%
GN 	 4 	 	 945 	 99.58%
AL 	 92.16% 	 99.58% 	 99.20%
```

# Panda
```
=========== GraspChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 53 	 	 0 	 	 100.00%
GN 	 257 	 	 690 	 72.86%
AL 	 17.10% 	 100.00% 	 74.30%
=========== GraspReacher ==============
	 PP 	 	 PN 	 	 1000
GP 	 53 	 	 0 	 	 100.00%
GN 	 38 	 	 909 	 95.99%
AL 	 58.24% 	 100.00% 	 96.20%
=========== FailMore ============== - new model
	 PP 	 	 PN 	 	 1000
GP 	 15 	 	 38 	 	 28.30%
GN 	 0 	 	 947 	 100.00%
AL 	 100.00% 	 96.14% 	 96.20%
=========== Balanced ============== - new model
	 PP 	 	 PN 	 	 1000
GP 	 42 	 	 11 	 	 79.25%
GN 	 9 	 	 938 	 99.05%
AL 	 82.35% 	 98.84% 	 98.00%
=========== Filtered ============== - new model
	 PP 	 	 PN 	 	 1000
GP 	 50 	 	 3 	 	 94.34%
GN 	 10 	 	 937 	 98.94%
AL 	 83.33% 	 99.68% 	 98.70%
```